In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import JointState
from std_msgs.msg import String
import time

In [2]:
joint_state = JointState()

class ARCSnakeListener(Node):
    '''
        Gets joint angle information from the snake
    '''
    def __init__(self):
        super().__init__('arcsnake_listener')
        self.subscription = self.create_subscription(
            JointState,
            '/arcsnake/joints_current',
            self.listener_callback,
            10)
        self.subscription  # prevent unused variable warning
        self.joint_publisher = self.create_publisher(JointState, '/arcsnake/joints_desired', 10)
        self.state_publisher = self.create_publisher(String, '/arcsnake/state', 10)

    def listener_callback(self, msg):
        global joint_state
        joint_state = msg

In [3]:
rclpy.init()

arcsnake_listener = ARCSnakeListener()
rclpy.spin_once(arcsnake_listener)

print(joint_state)


sensor_msgs.msg.JointState(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=0, nanosec=0), frame_id=''), name=['seg0_screw', 'seg0_uJoint2', 'seg1_uJoint1', 'seg1_screw', 'seg1_uJoint2', 'seg2_uJoint1', 'seg2_screw'], position=[1.3805554383275147, -0.1612208229955853, -0.08315700554274802, 3.086963848002371, -0.054089340182260644, 0.2350006504662547, 2.6145032194875055], velocity=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], effort=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])


In [4]:
rclpy.spin_once(arcsnake_listener)

In [5]:
arcsnake_listener.state_publisher.publish(String(data="start"))

In [6]:
arcsnake_listener.joint_publisher.publish(joint_state)

In [13]:
joint_state.position[1] -= 0.1
arcsnake_listener.joint_publisher.publish(joint_state)

In [14]:
factor = 8
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.velocity[0] = (factor*5)*i/number_of_steps  + 2
    joint_state.velocity[6] = -(factor*5)*i/number_of_steps  + 2
    
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)


In [11]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[2] += change_in_radians/number_of_steps
    joint_state.position[5] += -change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [19]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[2] += -change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [20]:
change_in_radians = 0.1
number_of_steps = 200
for i in range(number_of_steps):
    joint_state.position[5] +=  change_in_radians/number_of_steps
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

In [8]:
import math

In [9]:
start_point_2 = joint_state.position[2]
start_point_5 = joint_state.position[5]

In [15]:
max_angle = 0.5

for t in range(10000):
    f = 0.01
    joint_state.position[2] = start_point_2 + max_angle*math.sin(f*t)
    joint_state.position[5] = start_point_5 + max_angle*math.sin(f*t)
    arcsnake_listener.joint_publisher.publish(joint_state)
    time.sleep(0.01)

KeyboardInterrupt: 

In [16]:
arcsnake_listener.state_publisher.publish(String(data="off"))

In [17]:
# Destroy the node
arcsnake_listener.destroy_node()
rclpy.shutdown()